# Recomendador para series de anime. Entrega Final
__Emilio Lizama, Alonso Reyes, Juan-Pablo Silva__

## 1. Introducción

En la última década, la cultura japonesa ha a comenzado a despertar un interes creciente en occidente. Cada vez más gente está interesada en visitar Japón, leer su literatura, y como no, ver sus series de televisión animadas: anime.
El anime es un término usado para referirse a la animación japonesa, la cual puede tener tematicas infantiles, acción o temas más complejos, dirigidos a audiencias adultas.

Aun así con todo este aumento de personas interesadas y viendo anime, no existe alguna página web o aplicación que analice gustos y trate de recomendar series que probablemente serán de agrado de quien las buscó. Actualmente las páginas más populares para llevar registro de qué anime se han visto solo recomiendan anime de forma general por sus puntuaciones totales, o los clásicos "porque viste A te podría gustar B", cosas que son calculos básicos hechos acorde a los distintos géneros de cada serie y si coinciden con otras, de preferencia de mayor puntuación. Es por esto que nosotros creamos un sistema recomendador, que actúe como un predictor sobre si a cierto usuario le gustará o no cierto anime, y podrá generar recomendaciones personalizadas entorno a registros sobre cada usuario.

***

## 2. Datos

Los datos fueron obtenidos desde __[MyAnimeList](https://myanimelist.net/)__, una página que contiene una base de datos sobre todos los anime, junto a sus géneros, y puntajes promedios otorgados por usuarios. El principal propósito de esta página, es que sus usuarios sean capaces de guardar en sus "listas" los anime que han visto y que las califiquen asignando un puntaje del 1 al 10. De esta forma se modifican los puntajes promedio sobre los anime.

### 2.1 Obtención de datos

Para obtener estos datos implementamos un programa que recolecta información a través del código HTML de las páginas disponibles en MyAnimeList, es decir, un scraper o araña. Es importante confirmar que seguimos estrictamente las indicaciones fijadas por el archivo "robots.txt" y los terminos de servicio de MyAnimeList y no violamos ninguna ley en el proceso... mientras no distribuyamos ni usemos comercialmente los datos recolectados. Ademas se cuido de no sobrecargar los servidores de la pagina, por lo que tomo bastante tiempo obtener los datos (mas de 30 horas).

En total tenemos 14.007 anime y 4140 usuarios, además de esto, tenemos 43 generos, 462 estudios de animacion y 1.137 directores, entre otros datos.

El generar nosotros mismos los datos nos da acceso a decidir nosotros en qué formato, de qué manera y cómo queremos los datos, esto será explicado a continuación.

### 2.2 Descripción de los datos

Al usar el scraper, recolectamos los datos en 3 archivos: __anime__, __staff__ y __users__. El archivo __anime__ contiene información acerca de géneros, puntuaciones, nombres, entre otros mucho valores recolectados sobre todos los anime de la base de datos de MyAnimeList, esto es alrededor de 14.000 filas, lo que se reduce a 14.000 anime. 
El archivo __staff__ contiene informacion sobre las personas que trabajaron en cada anime, como directores, actores de voz, animadores, etc. Esta informacion corresponde a cada anime del archivo __anime__ del cual se tuviera la informacion.
El archivo __users__ contiene información sobre todos los anime que han visto, estan viendo y quieren ver de 4.140 usuarios aproximadamente. Ésto, un anime por fila por cada uno de los 4.140 usuarios, significa en alrededor de 900 mill filas. Es necesario decir que 4.140 no es ni cercanamente el total de usuarios presentes en el sitio, considerando que en todo momento se encuentran más de 8.000 usuarios online y el anime que registra más vistas contiene mas de 1 millón 200 mil, lo que significa que en el sitio existen al menos esa cantidad de usuarios. Dicho esto, la obtención de listas de usuarios al hacerse de acuerdo a los términos de servicio de MyAnimeList tomó un poco mas de 20 horas para los 4.140 usuarios, por lo que se estimó que usaremos esa cantidad para pruebas preliminares y exploración de los datos del proyecto.

Estos archivos contienen mucha información y en un formato no amigable con parsers para gráficos, ni bases de datos, por lo que reformateamos los datos. El archivo __anime__ se descompone en 3 archivos: __date__, __genre__, __general__ y __studios__. Los contenidos de cada uno de estos archivos corresponden a lo siguiente: 

- __general__: contiene información acerca del nombre, puntaje, favoritos, cantidad de episodios, fuentes de adaptación, formato de difusión, entre otras variables de cada anime en MyAnimeList.
- __date__: almacena las fechas de inicio y término de cada anime en el archivo __general__.
- __genre__: contiene cada género de cada anime listado en el archivo __general__.
- __studios__: contiene los estudios que trabajaron en la animacion de cada anime listado en __general__.

Para __users__ se filtro de tal manera que no contiene anime "plan to watch" ni "watching" de las listas de usuarios, ya que no consideramos que estos estados influyan realmente en los gustos de cada usuario, o al menos no se tiene la suficiente informacion para determinarlo. También, muchos datos sobre las listas de usuarios no tenian puntajes asignados a ciertos anime, por ello decidimos asignarle a cada uno el puntaje promedio total que calculamos. Además de esto, existen los anime en estado "dropped", es decir que el usuario dejó de verlo sin terminar, a estos anime rellenamos los valores incompletos con un puntaje de 3. Todas estas modificaciones fueron guardadas en un archivo __mal_filled__.

Para __staff__ se reformatearon los datos para que fuera mas facil leerlos posteriormente, ya que por simplicidad fueron creamos el archivo __anime | [lista de staff]__. Aqui simplemente se dejo una persona por fila, por cada anime y por cada rol que tuvo en este.

### 2.3 Exploración de los datos

Con los datos formateados y divididos en tablas que pueden ser usadas para graficos y tablas SQL hicimos algunos graficos para visualizar el general de datos que tenemos.

Antes que nada, importamos los módulos necesarios para visualizar.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib
matplotlib.style.use('ggplot')
%matplotlib notebook

Luego cargamos los archivos que contienen los datos que mostraremos:

In [ ]:
genre_data = pd.read_csv("genre.csv")
anime_data = pd.read_csv("general.csv")
user_data = pd.read_csv("mal_filled.csv")

Este es un merge de archivos especialmente lento, por lo que lo hacemos independientemente de los graficos.

In [ ]:
user_genre = pd.merge(genre_data, user_data, on='anime_id')

A continuación presentamos una estadística de géneros más vistos por intervalos de cantidad total de anime completados por el usuario. Esto es, cuáles son los géneros de anime más vistos dentro de usuarios que han visto cierta cantidad de anime.

In [ ]:
# Esto es particularmente lento
completed_data = user_genre[user_genre['anime_status'] == 'completed']
cnt = completed_data.groupby(['username'], as_index=False).agg({'anime_name': np.size}) # Cuantos ha visto cada usuario
cnt.columns.values[1] = 'count'
cnt_data = pd.merge(completed_data, cnt, on='username')

In [ ]:
intervals = []
genres = []
interval_size = 100
for i in range(20):
    c = cnt_data[(cnt_data['count'] > i*interval_size) & (cnt_data['count'] <= (i+1)*interval_size)]
    gb = c.groupby(['genre'], as_index=False).agg({'anime_name': np.size})
    gb.columns.values[1] = 'count'
    intervals.append("{}-{}".format((i*interval_size)+1, (i+1)*interval_size))
    top_three = [row['genre'] for index, row in gb.nlargest(5, ['count']).iterrows()]
    genres.append(top_three)

data = {'Genres': genres, 'Interval': intervals}
interval_df = pd.DataFrame(data)
cols = interval_df.columns.tolist()
cols = cols[-1:] + cols[:1]
interval_df = interval_df[cols]
interval_df

Como podemos ver, hay muy poca variación entre los géneros más vistos entre los distintos rangos de anime vistos. Si bien esto se sale un poco de lo que esperabamos, ya que esperabamos ver un poco más de variación entre los usuarios con más anime vistos, es logico que sea de esta manera, ya que sacamos un pequeño porcentaje de listas de usuarios de MyAnimeList. Ahora bien, la gran mayoría de rangos de usuarios tiene a "Comedia" como el género más visto, pero quizas esto pueda ser explicado por como se distribuyen los géneros de anime entre el total de anime que existen.

A continuación se presenta una estadística que muestra el total de anime con cierto género visto por cada usuario, versus el total de anime con ese género.

In [ ]:
#User count
total_user_genre = user_genre.groupby(by=['genre']).agg({'genre':np.size})

total_user_genre.sort_values(by='genre', ascending=False, inplace=True)
total_user_genre.rename(columns={'genre':'Total Count User'}, inplace=True)
total_user_genre.reset_index(inplace=True)
total_user_genre.rename(columns={'genre':'Genres'}, inplace=True)

#Total anime count
total_genre = genre_data.groupby(by=['genre']).agg({'genre':np.size})

total_genre.sort_values(by='genre', ascending=False, inplace=True)
total_genre.rename(columns={'genre':'Total Count Anime'}, inplace=True)
total_genre.reset_index(inplace=True)
total_genre.rename(columns={'genre':'Genres'}, inplace=True)

#Slow merge I guess
genre_user_anime_total = pd.merge(total_user_genre, total_genre, on='Genres')

genre_user_anime_total['Proportion'] = genre_user_anime_total['Total Count User']/genre_user_anime_total['Total Count Anime']

genre_user_anime_total.sort_values(by='Proportion', ascending=False, inplace=True)

genre_user_anime_total.head()

Como podemos ver, si bien comedia es el género más visto, también es el más abundante, por lo que al sacar una proporción simple entre cuantos se han visto versus cuantos hay, se obtiene que el más visto relativo a cuantos hay es el género de "Suspenso".

Como una explicación más visual de cómo se distribuyen los géneros vistos en las listas de los usuarios generamos el siguiente gráfico de torta.

In [ ]:
genres_counts = genre_data['genre'].value_counts() 
Otros = pd.DataFrame([genres_counts[12:].sum()], ["Other"])
genres_counts2 = genres_counts[:11].append(Otros)
genre_plot = pd.DataFrame(genres_counts2)

#pie = genre_plot.plot.pie(subplots=True, figsize=(6, 6),autopct='%1.1f%%')
pie = genre_plot.plot.pie(subplots=True, figsize=(8, 8),autopct='%1.1f%%',cmap='jet')

plt.title('Géneros de anime dentro de listas de usuarios', weight='bold', size=14)
plt.legend(bbox_to_anchor=(1.4,0.5), loc="center right", fontsize=10)
plt.subplots_adjust(left=0.1, bottom=0.3, right=0.7)
plt.ylabel('')
plt.show()

Así sabemos cómo se distribuyen los géneros en el total de anime y de anime vistos por cada usuario. Es importante recalcal nuevamente que poseemos solo un pequeño universo de datos de listas de usuarios, por lo que los resultados pueden variar cuando obtengamos más. Sin embargo, estimamos que las proporciones se mantengan casi estables por la distribución total de anime que hay, si hay más de X, es normal que se vea más X.

Ahora queremos saber como se distribuyen los puntajes de dados por cada usuario a los distintos anime, para ello hicimos la siguiente estadística.

In [ ]:
av_genre = user_genre.groupby(as_index=False, by=['genre']).agg({'user_score':np.mean})
av_genre.sort_values(by='user_score', ascending=False, inplace=True)
av_genre.rename(columns={'genre':'Genres', 'user_score':'Average user score'}, inplace=True)
av_genre.reset_index(drop=True, inplace=True)

av_genre.head()

Podemos ver que nuevamente aparece "Suspenso" como primero en la lista, en este caso aparece como el género con mejor puntuación promedio. Esto nos da una idea de cómo se encuentra distribuida el rango total de puntajes hechos por los usuarios. Para obtener la distribucion real de esto usamos el siguiente código.

In [ ]:
def scale_user_score(scores):
    return np.size(scores) / 1000

anime_average_score = user_data.groupby(by=['user_score']).agg({'user_score':scale_user_score})
anime_average_score.rename(columns={'user_score':'Count anime per 1000 users'}, inplace=True)
anime_average_score.reset_index(inplace=True)
anime_average_score.rename(columns={'user_score':'Score'}, inplace=True)

anime_temp_score = anime_data
anime_temp_score['int_score'] = anime_temp_score['score'].apply(np.round)
anime_total_score = anime_temp_score.groupby(by=['int_score']).agg({'int_score':np.size})
anime_total_score.rename(columns={'int_score':'Total anime count'}, inplace=True)
anime_total_score.reset_index(inplace=True)
anime_total_score.rename(columns={'int_score':'Score'}, inplace=True)


final_score = pd.merge(anime_average_score, anime_total_score, on='Score')

final_score


final_score.plot(kind='bar', x='Score', legend=False, title="Count of total user rated anime", subplots=True, layout=(1,2))

En el grafico anterior se muestran 2 graficos de barras, ambos representan como se distribuyen las puntuaciones de anime, pero sobre distintos objetos. En el primer gráfico se muestra como estan puntuados los anime en las listas de usuarios, como podemos ver, la gran mayoria de los anime fueron catalogados como 7.
En el caso del segundo gráfico, éste representa la distribución de los puntajes promedio totales entre todos los usuarios de MyAnimeList. Este puntaje esta incluido por cada anime que recolectamos. Como podemos ver, los puntajes de los usuarios que recolectamos se podrían considerar "inflados".
Se puede ver una irregularidad en el primer gráfico para el puntaje "3". Esto es porque nosotros manualmente rellenamos los anime en estado "dropped" que no tuvieran puntuacion con una clasificación de 3.

***

## 3. Metodología y pruebas

Ahora que tenemos conocimiento de cómo nuestros datos estan organizados, y las variaciones que existen, exploramos posibles algoritmos, modelos y librerías que pudieran ayudarnos a modelar nuestros datos de forma que pudiera precedir gustos.

Nuestra primera hipótesis es que si pudieramos clasificar usuarios en distintos grupos, podríamos hacer recomendaciones en base a los anime que X usuario no haya visto, pero que sí hayan visto y les hayan gustado a otros miembros del grupo. Esto se presenta en la siguientes secciones.


### 3.1 Matriz de interacción

Para todos los experimentos que presentaremos, se necesito de una tabla que contuviera informacion sobre anime y usuarios al mismo tiempo. Para esto se creo una matriz de interaccion, es decir, un mappeo entre lo que cada usuario ha visto y que anime existen. Cada fila de la matriz representa a un usuario, por lo que hay del orden de 4 mil filas. Y cada columna representa a un anime, donde el valor que se encuentra en cada celda de la matriz corresponde al puntaje que ese usuario le dio al anime correspondiente. En caso de no haberlo visto el valor es 0. De esta forma tenemos una matriz de 4.140 por 14.007, la cual es sumamente dispersa, lo que significa que gran parte de la matriz esta rellena de ceros, y solo algunas celdas son distintas de 0. Aqui en vez de guardar la matriz completa, solo guardamos las coordenadas distintas de 0, de tal forma que primero se cree una matriz de solo ceros para luego reemplazar los valores correspondientes. Para esto se utilizaron las matrices sparse de scipy. El codigo sobre como se construyo esta matriz se encuentra en __[parser.py](https://github.com/juanpablos/anime-recommender/blob/master/src/parser.py)__.


### 3.2 Dimensionalidad

El primer problema con el que chocamos rápidamente al proponernos clasificar usuarios, fue la dimensionalidad de nuestros datos. Con más de 14 mil anime, se tienen más de 14 mil dimensiones para considerar. Además, la matriz de interacción es sumamente dispersa, ya que los usuarios en promedio han visto menos de 200 anime.

Para datos dispersos como los nuestros, aplicar PCA no trae los mismos resultados que traería para una matriz más densa. Por esta razón, una solución que usamos fue LSA (latent semantic analysis) que nos entrega una matriz con vectores latentes, con una dimension que nosotros especificamos. Esto es principalmente porque LSA no centra los datos antes de computar el SVD (singular value descomposition). Esto permite que funcione mucho más rápido con la representación de matrices dispersas de scipy.


### 3.3 Clustering

Aqui proponemos usar clustering para agrupar usuarios y determinar gustos. Sin embargo, estos experimentos no son usados independientemente, sino mas bien usados con el intento de mejorar el modelo final en 3.4.

#### 3.3.1 K-means

Uno de los primeros pasos que tomamos fue usar K-Means para clasificar nuestros datos, en este caso, para clasificar usuarios. En este contexto se hicieron 3 pruebas. Una con las 14 mil dimensiones de la matriz dispersa, es decir con los datos originales; otra con una reducción de dimensiones de 14 mil a mil, y la ultima reduciendo las dimensiones a 100. El porqué se redujo a mil es porque creemos que si bien 14 mil anime es demasiado, y nadie los ha visto todos, y muy buena parte de ellos tampoco son populares, mil anime no es mucho. Hay muchos usuarios que han visto más de 1.000 series, por lo que consideramos que tener mil valores representativos es una buena medida.

Como no sabemos realmente cuantos grupos de usuarios queremos, usamos el elbow method, que se basa en probar una serie de "K"s, número de clusters, luego calcular la inercia, o la suma del error cuadrático, y graficar este error decrecer. La idea es escojer el K que hace disminuir más el error antes de que este empiece a disminuir menos abruptamente. Como tenemos datos tan dispersos, cláramente el error no se estabilizará fácilmente, pero nos podrá dar una idea de qué valor de K es uno útil para nuestro análisis.

Los graficos obtenidos para el numero original de dimensiones (14 mil), y para mil dimensiones son los siguientes:

![kmeans](elbow_kmeans.png)
![kmeans_lsa](elbow_kmeans_lsa.png)

Como se puede ver, el error no deja de disminuir. Esto es lógico ya que el error se hace 0 solo cuando K=número de elementos. Respaldado parcialmente por los gráficos, decidimos utilizar un K=50, con el cual K-Means le asignó a cada usuario un grupo. No validamos matematicamente los clusters obtenidos, pero inspecciones manuales sobre algunos grupos de clusters selectos resultó en agrupaciones correctas de usuarios con gustos similares. Por supuesto, este tipo de evaluación no tiene respaldo formal, pero es una señal de que el método en efecto tiene resultados.

Una vez sabiamos el numero de clusters a utilizar creamos funciones que corrieran K-Means y guardaran en un archivo el cluster asociado a cada usuario.

#### 3.3.2 Ki-Means

Basado en K-means, Ki-means es nuestra implementación del algoritmo para hacer clustering sobre datos dispersos. Nuestra hipotesis para hacer esto es la siguiente: asumamos existen solo 3 anime tal que cada anime es representado por su índice en un vector de 3 valores. Digamos Pedro puso un 7 al primer anime, no ha visto el segundo y puso un 1 al tercero, esto sería (7, 0 ,1). Luego tenemos a Pablo que puso los siguientes puntajes (7, 10 , 0), es decir no ha visto el tercero. Finalmente esta Alejandro con (2, 10, 5). En un escenario como este K-means con K=2 clasificaría juntos a Pablo y Alejandro, ya que su distancia euclideana es efectivamente menor. Sin embargo, nosotros proponemos que los que realmente se parecen más son Pedro con Pablo, ya que no tenemos información sobre si a Pedro le gustará o no el segundo anime, y tampoco tenemos información sobre el 3er anime para Pablo, pero ambos pusieron un 7 al primer anime, por lo que su distancia sería 0.

Es basado en esta hipótesis que implementamos K-means de forma que considerara solo variables comunes entre puntos, salvo el centroide, en el cual se tomaban todas sus variables en cuenta. En pocas palabras esta implementación no considera la variables que tiene 0 en los puntos del dataset, es decir, no considera los anime que los usuarios no han visto cuando interactúan entre ellos. El código de la implementación se encuentra en __[ki_means.py](https://github.com/juanpablos/anime-recommender/blob/master/src/ki_means.py)__. La implementacion no contiene ningun tipo de optimizacion, por lo que toma un tiempo elevado para agrupar en clusters. Por esta razon, en vez de utilizar el elbow method para seleccionar el numero de clusters, simplemente elegimos 50, como en K-Means.

### 3.4 Factorization Machine

Las Factorization Machines son un concepto relativamente nuevo introducido el 2010 por Steffen Rendle [1]. Las FM son un modelo que permite simular la gran mayoría de los modelos de factorización de matries mediante feature engineering, que básicamente es entender sobre el dominio en que se esta trabajando para crear características para entrenar modelos de machine learning. Según su mismo inventor, las FM son comparables a los SVMs (support vector machine) con un kernel polinomial, y la factorizacion de matrices. La diferencia mayor yace en que la FM tiene una complejidad de tiempo lineal, mietras que el SVM es cuadratico para el mismo input. Ademas la FM funciona muy bien con datos dispersos a cambio de no tener buenos resultados con datos densos, que es lo contrario al SVM.

La parte importante que se aplica a nuestro problema de recomendar series de anime, es que la FM pueden considerar metadata, ademas de solo la matriz de interaccion, creando un modelo hibrido. La mayoría de los sistemas recomendadores toman una matriz (usuario, item, puntuación), pero hay veces en las que también se tienen datos externos a esas interacciones, como pueden ser tags, géneros de música, pre clasificación de usuarios, etc. Esta es justamente nuestra situación, tenemos géneros, múltiples, para cada anime, tambien estudios de animacion y directores. Además podríamos considerar edad, sexo y país de residencia de los usuarios a evaluar, lo cual se convierte en más carácteristicas extra, que llamaremos metadata.

Aquí utilizamos LightFM [2], una librería de Python para trabajar con factorization machines que implementa un modelo híbrido con feedback explicito e implicito basada en el ranking de items. Para el parseo de datos, generacion de matrices y selección de caracteristicas se crearon una serie de funciones auxiliares que crean una capa de abstraccion para la facil manipulacion de datos, esto se encuentra en __[data_utils.py](https://github.com/juanpablos/anime-recommender/blob/master/src/data_utils.py)__.


#### 3.4.1 Testing

Para testear el modelo entrenado existen 2 maneras. Una es teorica, usando tecnicas de evaluacion, que es lo que usaremos. Sin embargo, las FM y los sistemas recomendadores en general tienden a ser evaluadas empiricamente, usando usuarios que califiquen al modelo, en vez de usar una metrica teorica para hacerlo. Esto ya que con tantos itemes que recomendar, no es realmente posible verificar si efectivamente al usuario le gustara o no una recomendacion si no es preguntandole a el o ella personalmente.

El conjunto de testing que elegimos para verificar teoricamente el modelo consisten en remover el 25% de los anime vistos por cada usuario aleatoriamente y agregarlos en otra matriz, dejando como "no visto" el anime removido en la matriz original. Es decir, tenemos una matriz con el 25% de los anime visto por cada usuario que utilizaremos para verificar el modelo.


#### 3.4.2 Metricas

Para evaluar el modelo requerimos de metricas que determinen la correctitud de este. LightFM provee de 2 funciones representativas sobre modelos recomendadores que ayudan a tener una idea de que ocurre con las recomendaciones y que tan acertadas son. Las metricas usadas son las siguientes:

- __AUC score__: se refiere al area bajo la curva de la matriz de confucion para distintos umbrales, y se interpreta como que tan probable es que un verdadero positivo quede rankeado mas "arriba" (o antes) que un falso negativo. Esto no considera que tan antes debe estar rankeada, sino que solo si es antes o despues.
- __Precision at k__: se refiere al numero de items que son verdaderos positivos dentro de los k primeras posiciones del ranking. Esto signfica que pido al recomendador que me diga el top K, luego comparo esos K items y veo si se encuentran en el set de prueba, esto corresponde a la cantidad de correctas partido por K. Se puede notar que para un numero de items arbitrariamente grande la precision deja de tener sentido ya que es muy dificil que seleccione exactamente algun item en el set de prueba.

Para las evaluaciones utilizamos un k=10 para el precision at k.


#### 3.4.3 Resultados

DECIR QUE SE USARON NADA, GENRE, DIRECTORS, STUDIOS. LUEGO GENRE ES MEJOR ASI QUE SE UTILIZO CON LOS RESULTADOS DE KMEANS.

TABLAS CON RESULTADOS

***

## 4. Conclusiones

BUEN MODELO
METADATA DE GENEROS AUMENTA EL ACCURACY
KIMEANS NO FUE BUENO
CLUSTERING NO APORTA DIRECTAMENTE AL ACCURACY, SIN EMBARGO SIRVE PARA CLASIFICAR A LOS USUARIOS Y EXTENDER EL MODELO A USUARIOS NO REGISTRADOS DENTRO DEL MODELO, ex NUEVOS USUARIOS (SIN TENER QUE RE ENTRENAR TODO)

***

## 5. Trabajo Futuro

OBTENER MAS USUARIOS, SOLO TENEMOS 4000 DE MILLONES
HACER ESTUDIO DE USUARIOS (VERIFICACION EMPIRICA)
ENSAMBLE DE MODELOS O METADATA PARA MEJORAR EL ACCURACY, O PERMITIR FILTROS O RECOMENDACIONES BAJO CRITERIOS
BORRAR KIMEANS Y CAMBIAR FUNCION DE DISTANCIA A KMEANS PARA REEMPLAZARLO, UTILIZAR FUNCION QUE CONSIDERE 0 Y x (OTRO VALOR) NO COMO DISTANCIA x, SINO QUE DISTANCIA 0. ex COSENO.
FUNCION DE EVALUACION PROPORSIONAL A LA PUNTUACION DEL USUARIO, PRECISION AT K PERO CON PROPORSION.

## Referencias
[1] Rendle, S. (2010). Factorization Machines.. In G. I. Webb, B. Liu, C. Zhang, D. Gunopulos & X. Wu (eds.), ICDM (p./pp. 995-1000), : IEEE Computer Society. ISBN: 978-0-7695-4256-0 

[2] Kula, M. (2015). Metadata Embeddings for User and Item Cold-start Recommendations.. In T. Bogers & M. Koolen (eds.), CBRecSys@RecSys (p./pp. 14-21), : CEUR-WS.org. 